<a href="https://colab.research.google.com/github/jenieto/pollen-classification/blob/master/Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montamos Google Drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Extract data
!mkdir -p /data
!tar xvzf "/content/drive/My Drive/Datasets/anuka1200.tar.gz" --directory /data

In [ ]:
# Create datasets
import tensorflow as tf

training_size = 0.8
validation_size = 0.2
image_height = 50
image_width = 50
image_channels = 3
image_size = (image_height, image_width, image_channels)
directory = '/data/anuka1200'

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=directory,
    subset="training",
    validation_split=training_size,
    seed=123,
    image_size=(image_height, image_width))

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=directory,
    subset="validation",
    validation_split=validation_size,
    seed=123,
    image_size=(image_height, image_width))

class_names = train_dataset.class_names
print('Class Names', class_names)

In [ ]:
# Explore data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [5]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


def create_models():
  model_0 = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=image_size),
    layers.Flatten(),
    layers.Dense(2, activation='sigmoid')
  ])

  model_1 = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=image_size),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='sigmoid')
  ])

  model_2 = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=image_size),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='sigmoid')
  ])

  return {
      'model_0': model_0,
      'model_1': model_1,
      'model_2': model_2
  } 

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Compile models
def compile_models(models_dict):
  for model_name, model in models_dict.items():
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']) 
    # print(f'{model_name} summary:')
    # model.summary()

  
def model_callbacks(model_name):
  filepath_mdl = 'model.h5'
  checkpoint = ModelCheckpoint(filepath_mdl, monitor='val_loss', verbose=1, save_best_only=True) # Va guardando los pesos tras cada época
  log_dir = f"logs/fit/{model_name}/"# + datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True) # Para graficado de las estadísticas durante el entrenamiento
  earlystopping = EarlyStopping(patience=20, verbose=1) # Detiene el entrenamiento prematuramente si validation accuracy lleva sin aumentar varias épocas
  return [checkpoint, tensorboard, earlystopping]

In [17]:
# Train models
def train_models(models_dict):
  epochs=100
  for model_name, model in models_dict.items():
    print('-----------------------------------------------------------------------')
    print(f'Fitting model {model_name}')
    history = model.fit(
      train_dataset,
      validation_data=validation_dataset,
      epochs=epochs,
      callbacks=model_callbacks(model_name))


In [ ]:
# Execute
models = create_models()
compile_models(models)
train_models(models)

In [ ]:
# Start Tensorboard
%load_ext tensorboard
%tensorboard --logdir logs